# HyLoRADA Unified (v0.3.0)

Parameter-efficient fine-tuning for long-context LLMs.

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate tqdm

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Test imports
from hylorada import HyLoRADAConfig, HyLoRADAModel
config = HyLoRADAConfig(lora_rank=8)
print("Config:", config.get_component_status())

## Run Benchmark

In [ ]:
!python run_benchmark.py --methods lora hylorada --epochs 3 --num_train 500

## Train on Code

In [ ]:
!python run_code_task.py --epochs 3 --rank 8

## Results

In [ ]:
import json, os, glob

for f in glob.glob('./**/benchmark*.json', recursive=True):
    print(f"\n=== {f} ===")
    data = json.load(open(f))
    if 'results' in data:
        for m, r in data['results'].items():
            if 'perplexity' in r:
                print(f"{m}: PPL={r['perplexity']:.2f}")